In [1]:
from gpt_lm import GPTWrapper, construct_model, SupervisedDataset
import transformers
import torch
import pandas as pd
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader, Sampler

"Собираем" модель:

In [2]:
model, kwargs = construct_model()

In [10]:
kwargs

{'max_new_tokens': 100,
 'num_beams': 3,
 'early_stopping': True,
 'no_repeat_ngram_size': 2}

Подгружаем датасет для обучения:

In [4]:
sft_df = pd.read_parquet('datasets/argilla_df.parquet')

In [5]:
sft_df

,prompt,response-1,response-2,category,original_response,external_id
0,What is Depreciation,What is Depreciation – 10 Important Facts to K...,What is Depreciation on a Car?\nDepreciation i...,open_qa,Depreciation is the drop in value of an asset ...,518
1,What do you know about the city of Aberdeen in...,"Aberdeen, the city in which I've lived for the...","As an AI language model, I don't have personal...",open_qa,Aberdeen is a city located in the North East o...,351
2,Describe thunderstorm season in the United Sta...,Describe thunderstorm season in the United Sta...,Describe thunderstorm season in the United Sta...,information_extraction,Thunderstorm season in the United States and C...,10567
3,"When did Peloton IPO?\nOn September 26, 2019, ...",When did Peloton IPO?\nPeloton launched its in...,When did Peloton IPO?\nPeloton IPO'd on May 26...,closed_qa,Peloton became a public company via an initial...,12412
4,What is the best way to answer an interview qu...,The best way to answer an interview question m...,Some of the best ways to answer an interview q...,creative_writing,The first recommended step is to ask clarifyin...,2521
...,...,...,...,...,...,...
7396,How do i accept the change,How do i accept the change in my life\nAccepti...,I's a great opportunity to improve. The only t...,brainstorming,Embrace the change and see the difference,15010
7397,Extract the teams that the footballer Sócrates...,Extract the teams that the footballer Sócrates...,Extract the teams that the footballer Sócrates...,information_extraction,"Brazil, Botafogo-SP, Corinthians, Fiorentina",9970
7398,Without quoting directly from the text give me...,Without quoting directly from the text give me...,Without quoting directly from the text give me...,summarization,"Brendon Small is a stand-up comedian, Creator...",14205
7399,Is Killing is Sin ? Is it ture,Is Killing is Sin ? Is it ture?\nKilling can b...,Is Killing is Sin ? Is it ture?\nKilling is no...,brainstorming,Killing a human being should not be sin becaus...,11253


In [6]:
train_batch = SupervisedDataset(
    dataset=sft_df[:7000][['prompt', 'original_response']].values,
    tokenizer=model.tokenizer,
)

In [7]:
train_batch_loader = DataLoader(
    dataset=train_batch,
    batch_size=1,
    shuffle=False,
)

Обучаем модель на полученном датасете.
P.S. писал на мак оси, поэтому device = 'mps', cuda тут не поддерживается. Если сидите с винды или линукса пропишите device = 'cuda'

In [9]:
from torch import nn
device = 'mps'
model.model.to(device)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.model.parameters(), lr= 0.0000001)
N_ITERATIONS = 200
cur_iteration = 0
loss_values = []

for x in train_batch_loader:
    if cur_iteration == N_ITERATIONS:
      break

    model.model.train()
    #################
    input_tokens = x['input_ids'].to(device)
    attention_mask = x['input_att_mask'].to(device)
    labels = x['labels'].clone().to(device)
    out_logits = model.model(input_ids=input_tokens, attention_mask=attention_mask).logits

    ### не затираем токены начала и конца, чтобы модель училась останавливаться ###
    labels[labels == 60000] = -100

    loss_value = loss(out_logits.permute(0, 2, 1), labels)
    #################
    print(f"Loss value: {loss_value.item()}")
    loss_value.backward()
    optimizer.step()

    model.model.eval()
    cur_iteration += 1
    loss_values.append(loss_value.cpu().detach().numpy())

Loss value: 3.920064926147461
Loss value: 3.1432785987854004
Loss value: 0.6254844069480896
Loss value: 1.5423660278320312


KeyboardInterrupt: 

Сохраняю конфиг модельки:

In [18]:
torch.save(model.model.state_dict(), 'models/gpt_lm/model.pkl')